In [ ]:
import ROOT
import numpy as np
import pandas as pd

In [ ]:
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from bokeh.models import *
from bokeh.io import output_notebook
#from bokeh.plotting import figure
output_notebook()

In [ ]:
from RootInteractive.MLpipeline.NDFunctionInterface import DataContainer, Fitter

## Load AliRoot libraries

In [ ]:
ROOT.gSystem.Load("libSTAT")
ROOT.gSystem.Load("libCDB")
ROOT.gSystem.Load("libESD")

## Load OCDB

In [ ]:
ROOT.gROOT.LoadMacro("$ALICE_PHYSICS/PWGPP/TPC/macros/ConfigOCDB.C")  
ROOT.ConfigOCDB(129160,"local:///cvmfs/alice-ocdb.cern.ch/calibration/data/2010/OCDB/")

## Create example vector - default in python is list 
* we need C++ vectors

In [ ]:
materialCorr=0
nPoints=100

# initialize coordinates, momentum and covariance matrix as TVector to use c++ functions
xyz=ROOT.TVectorD(3)
pxyz=ROOT.TVectorD(3)
cv=ROOT.TVectorD(21)
covarAdd=ROOT.TVectorD(15)
covarAdd[0]=1
covarAdd[2]=1.
covarAdd[5]=1./(15.*15.)
covarAdd[9]=3.*3./(15.*15.)
covarAdd[14]=1*1

# load TRD geometry
trdGeom = ROOT.AliTRDgeometry

#get position of layers
layers=[]
for i in range(7):
    layers.append(trdGeom.GetTime0(i))
layers.append(400)  
layers.append(420)

# initializations
pointsOut=ROOT.TMatrix(8,3)
pointsIn=ROOT.TMatrix(8,3)
massEl=0.0005
point=ROOT.TVectorD(2)
covPoint=ROOT.TVectorD(3)
#resolution
covPoint[0]=0.05*0.05
covPoint[2]=1*1

In [ ]:
# test track
track0  = ROOT.AliExternalTrackParam(xyz.GetMatrixArray(),pxyz.GetMatrixArray(),cv.GetMatrixArray(),1)
layers

In [ ]:
ROOT.AliTrackerBase.PropagateTrackToBxByBz?

In [ ]:
# generate track in trd direction
def generateTrack():
    # generate intial track params and points along trajectory
    #tanA=np.tan(np.pi/9)*np.random.uniform() # central trd interaction
    tanA=np.tan(np.pi/9)*0.5
    pxyz[0]=np.random.uniform()+0.10
    pxyz[1]=pxyz[0]*tanA
    pxyz[2]=0
    xyz[0]=trdGeom.GetTime0(0)
    xyz[1]=xyz[0]*tanA
    xyz[2]=0
    
    # initial parameters as AliExternalTrackParam
    trackIn  = ROOT.AliExternalTrackParam(xyz.GetMatrixArray(),pxyz.GetMatrixArray(),cv.GetMatrixArray(),1)
    trackOut  = ROOT.AliExternalTrackParam(trackIn) 
    
    # propagate track through detector and obtain trackOut
    for layer in range(8):
        # Bool_t AliTrackerBase::PropagateTrackToBxByBz(AliExternalTrackParam* track, Double_t x, Double_t m, Double_t maxStep, Bool_t rotateTo = kTRUE, Double_t maxSnp = 0.80000000000000004, Int_t sign = 0, Bool_t addTimeStep = kFALSE, Bool_t correctMaterialBudget = kTRUE)
        ROOT.AliTrackerBase.PropagateTrackToBxByBz(trackOut,layers[layer],  massEl, 1, 0, 0.8,0,0, materialCorr)
        pointsOut[layer,0] = trackOut.GetX()
        pointsOut[layer,1] = trackOut.GetY()
        pointsOut[layer,2] = trackOut.GetZ()
        
    # fit track from tof to inner trd layer and update parameters
    trackFit  = ROOT.AliExternalTrackParam(trackOut)
    trackFit.AddCovariance(covarAdd.GetMatrixArray())
    trackFits=[0,0,0,0,0,0,0,0,0,0]
    for layer in reversed(range(8)):
        # Bool_t AliTrackerBase::PropagateTrackToBxByBz(AliExternalTrackParam* track, Double_t x, Double_t m, Double_t maxStep, Bool_t rotateTo = kTRUE, Double_t maxSnp = 0.80000000000000004, Int_t sign = 0, Bool_t addTimeStep = kFALSE, Bool_t correctMaterialBudget = kTRUE)
        ROOT.AliTrackerBase.PropagateTrackToBxByBz(trackFit,layers[layer],  massEl, 1, 0, 0.8,0,0, materialCorr)
        point[0]=pointsOut(layer,1)+np.random.normal()*0.05
        point[1]=pointsOut(layer,2)+np.random.normal()*1
        trackFit.Update(point.GetMatrixArray(),covPoint.GetMatrixArray())
        pointsIn[layer,0] = trackOut.GetX()
        pointsIn[layer,1] = trackOut.GetY()
        pointsIn[layer,2] = trackOut.GetZ()
        trackFits[layer]=ROOT.AliExternalTrackParam(trackFit)
        
    # return initial conditions, track fit parameters
    return trackIn, trackFits, pointsOut

In [ ]:
np.tan(np.pi/9)*0.5

In [ ]:
trackIn, trackFits, points=generateTrack()

In [ ]:
trackIn.Print()

In [ ]:
# generate 100 tracks and obtain pt, pt after fit and sigma, layer, position
NTracks = 10000
Pt = []
Pt_fit = []
Sigma1Pt = []
LayerN = []
Xpos = []
Ypos = []
Zpos = []
TrackIndex = []

for i in range(NTracks):
    trackIn, trackFits, points=generateTrack()
    for j in range(4):
        TrackIndex.append(i)
        Pt.append(trackIn.Pt())
        Pt_fit.append(trackFits[j].Pt())
        Sigma1Pt.append(np.sqrt(trackFits[j].GetSigma1Pt2()))
        LayerN.append(j)
        Xpos.append(pointsIn(j,0))
        Ypos.append(pointsIn(j,1))
        Zpos.append(pointsIn(j,2))
        
layerdict = {"TrackIndex" : TrackIndex, "Pt" : Pt, "Pt_fit" : Pt_fit, "Sigma1Pt" : Sigma1Pt, "LayerN" : LayerN, "Xpos" : Xpos, "Ypos" : Ypos, "Zpos" : Zpos}
df_layers = pd.DataFrame(layerdict)

In [ ]:
df_layers.head(5)

In [ ]:
#fit resolution 
varFit='Sigma1Pt'
variableX= ['Pt_fit',"LayerN", "Ypos"]
npoints = NTracks*4
dataContainer = DataContainer(df_layers, variableX, varFit, [npoints//2,npoints//2])
fitter = Fitter(dataContainer)
fitter.Register_Method('RF','RandomForest', 'Regressor', n_estimators=100, max_depth=10)
fitter.Register_Method('RF200','RandomForest', 'Regressor', n_estimators=200, max_depth=10)
fitter.Fit()

In [ ]:
df_layers = fitter.AppendOtherPandas("RF",df_layers)
df_layers = fitter.AppendOtherPandas("RF200",df_layers)
fitter.AppendStatPandas("RF",df_layers,"Sigma1Pt")
fitter.AppendStatPandas("RF200",df_layers,"Sigma1Pt")
print("")

In [ ]:
figureArray = [
    [['Pt'], ['Sigma1Pt*Pt'], { "colorZvar":"LayerN"}], 
    [['LayerN'], ['Sigma1Pt*Pt'], { "colorZvar":"Pt"}], 
    ['table', {'include':'.*'}],
    {"size":10}
]
figureLayout=[
    [0, {'commonY':0,'y_visible':2,'plot_height':100}],
    [1, {'commonY':0,'y_visible':2,'plot_height':100}],
    [2, {'plot_height':20}],
    {'plot_height':100,'sizing_mode':'scale_width'}
]
widgetArray=[
    ['range', ['Pt']],
    ['range', ['LayerN']],
     ['range', ['TrackIndex']],
    #['slider','F', ['@min()','@max()','@med','@min()','@median()+3*#tlm()']], # to be implmneted
]
widgetLayout=[ [0,1,2], {'sizing_mode':'scale_width'} ]
tooltips = [("Pt", "(@Pt)"), ("TrackIndex", "(@TrackIndex)"), ("LayerN", "(@LayerN)")]  

In [ ]:
output_file("test_toyTracking.html")
bokehDrawSA.fromArray(df_layers.head(500), "Pt>0", figureArray, widgetArray, layout=figureLayout, tooltips=tooltips,widgetLayout=widgetLayout,sizing_mode="scale_width")

In [ ]:
# generate 100 tracks and obtain pt, pt after fit and sigma
NTracks = 100

pt = []

pt_fit_list = []
Sigma1Pt2_fit_list = []
for i in range(4):
    pt_fit = []
    Sigma1Pt2_fit = []
    pt_fit_list.append(pt_fit)
    Sigma1Pt2_fit_list.append(Sigma1Pt2_fit)
    
for i in range(NTracks):
    trackIn, trackFits, points=generateTrack()
    pt.append(trackIn.Pt())
    for j in range(4):
        pt_fit_list[j].append(trackFits[j].Pt())
        Sigma1Pt2_fit_list[j].append(np.sqrt(trackFits[j].GetSigma1Pt2()))
        

df_dict = {'pt':pt}

for i in range(4):
    df_dict.update({"pt_fit_"+str(i) : pt_fit_list[i]})
    df_dict.update({"Sigma1Pt"+str(i) : Sigma1Pt2_fit_list[i]})
    
df = pd.DataFrame(df_dict)

In [ ]:
df.head(5)

In [ ]:
figureArray = [
    [['pt'], ['Sigma1Pt0*pt']],
    [['pt'], ['Sigma1Pt1*pt']],
    [['pt'], ['Sigma1Pt2*pt']],
    [['pt'], ['Sigma1Pt3*pt']],
    
    ['table', {'include':'.*'}],
    {"size":10}
]
figureLayout=[
    [0,1, {'commonY':3,'y_visible':2,'plot_height':200}],
    [2,3,{'commonY':3, 'plot_height':200}],
    [4, {'plot_height':50}],
    {'plot_height':100,'sizing_mode':'scale_width'}
]
widgetArray=[
    ['range', ['pt']],
    #['slider','F', ['@min()','@max()','@med','@min()','@median()+3*#tlm()']], # to be implmneted
]
widgetLayout=[ [0], {'sizing_mode':'scale_width'} ]
tooltips = [("pt", "(@pt)"),("s0", "@Sigma1Pt0"), ("s1", "@Sigma1Pt3")]  

In [ ]:
output_file("test_toyTracking.html")
bokehDrawSA.fromArray(df, "Sigma1Pt3*pt<1", figureArray, widgetArray, layout=figureLayout, tooltips=tooltips,widgetLayout=widgetLayout,sizing_mode="scale_width")